In [54]:
import os
from pathlib import Path

import pandas as pd

LEVEL = 12

music_master = pd.read_csv("../../data/notes.csv", na_values="-",
                           usecols=[
                               "version", "title", "hyperNotes", "anotherNotes", "leggendariaNotes",
                               "hyperLevel", "anotherLevel", "leggendariaLevel"
                           ])
filtered_master = music_master[(music_master["hyperLevel"] == LEVEL) |
                               (music_master["anotherLevel"] == LEVEL) |
                               (music_master["leggendariaLevel"] == LEVEL)]

versions = ["27", "28", "29"]
for version in versions:
    os.makedirs("../../data/data_preparation/" + version, exist_ok=True)
    dir_path = Path("../../data/" + version)
    file_list = dir_path.glob("*.csv")
    
    concat = None
    for file in file_list:
        if "00_全国" in file.name:
            continue
        
        ranker_scores = pd.read_csv("../../data/" + version + "/" + file.name,
                                    usecols=[
                                        "タイトル", "HYPER EXスコア", "HYPER DJName", "HYPER 都道府県",
                                        "ANOTHER EXスコア", "ANOTHER DJName", "ANOTHER 都道府県",
                                        "LEGGENDARIA EXスコア", "LEGGENDARIA DJName", "LEGGENDARIA 都道府県"
                                    ])
        joined = pd.merge(filtered_master, ranker_scores, left_on="title", right_on="タイトル", how="inner")
        
        if concat is None:
            concat = joined
        else:
            concat = pd.concat([concat, joined])
    
    concat["HYPER 順位"] = concat.groupby(["title"])["HYPER EXスコア"].rank(ascending=False)
    concat["ANOTHER 順位"] = concat.groupby(["title"])["ANOTHER EXスコア"].rank(ascending=False)
    concat["LEGGENDARIA 順位"] = concat.groupby(["title"])["LEGGENDARIA EXスコア"].rank(ascending=False)
    concat.sort_values(["title"]).to_csv("../../data/data_preparation/" + version + "_concat.csv", index=False)

total = []
for version in versions:
    csv = pd.read_csv("../../data/data_preparation/" + version + "_concat.csv")
    total.append(csv)
    
pd.concat(total).sort_values(["title"]).to_csv("../../data/data_preparation/concat.csv", index=False)